In [231]:
import os 
os.getcwdb()
os.listdir("data")

['sales_data.csv', 'customer_data.csv', 'product_data.csv', 'store_data.csv']

In [232]:
import pandas as pd
import numpy as np

# load datasets from the data folder
customer = pd.read_csv("data/customer_data.csv")
product = pd.read_csv("data/product_data.csv")
sales = pd.read_csv("data/sales_data.csv")
store = pd.read_csv("data/store_data.csv")

# preview the dataset
sales.head()




,transaction_id,date,product_id,store_id,customer_id,quantity,discount,returned
0,T0000001,2023-08-02,P004681,S004,C010043,3,0.0,0
1,T0000002,2020-08-16,P006662,S003,C022472,1,0.0,0
2,T0000003,2020-02-21,P043402,S005,C016135,4,0.0,0
3,T0000004,2024-02-27,P029875,S003,C005605,3,NaN,1
4,T0000005,2021-07-17,P002476,S003,C005857,4,0.0,0


In [233]:
# QUICK SANITY CHECK; We confirm the data actually loaded and looks correct.

print("CUSTOMER:", customer.shape)
print("PRODUCT :", product.shape)
print("SALES    :", sales.shape)
print("STORE   :", store.shape)

sales.head()

CUSTOMER: (25000, 5)
PRODUCT : (50000, 8)
SALES    : (50000, 8)
STORE   : (5, 4)


,transaction_id,date,product_id,store_id,customer_id,quantity,discount,returned
0,T0000001,2023-08-02,P004681,S004,C010043,3,0.0,0
1,T0000002,2020-08-16,P006662,S003,C022472,1,0.0,0
2,T0000003,2020-02-21,P043402,S005,C016135,4,0.0,0
3,T0000004,2024-02-27,P029875,S003,C005605,3,NaN,1
4,T0000005,2021-07-17,P002476,S003,C005857,4,0.0,0


In [234]:
# Structure + Missing Values Summary;verify columns, datatypes, and missing values.
for name, df in {
    "customers": customer,
    "products": product,
    "sales": sales,
    "stores": store
}.items():
    print(f"\n--- {name.upper()} ---")
    print(df.info())
    print("\nMissing values per column:")
    print(df.isna().sum())


--- CUSTOMERS ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  25000 non-null  object
 1   age          25000 non-null  int64 
 2   gender       25000 non-null  object
 3   city         25000 non-null  object
 4   email        24504 non-null  object
dtypes: int64(1), object(4)
memory usage: 976.7+ KB
None

Missing values per column:
customer_id      0
age              0
gender           0
city             0
email          496
dtype: int64

--- PRODUCTS ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   product_id  50000 non-null  object 
 1   category    50000 non-null  object 
 2   color       49010 non-null  object 
 3   size        50000 non-null  object 
 4   season      5000

In [235]:
# CLEANING SALES: PARSE DATE

# Dates often load as text; convert to datetime so we can group by month/year.

sales["date"] = pd.to_datetime(sales["date"], errors="coerce")

# Check if any dates failed to parse (became NaT)
sales["date"].isna().sum()

0

In [236]:
# Validate Numeric Logic (Quality checks BEFORE editing)

# SALES VALIDATION: NUMERIC LOGIC
# =========================
# Before analysis we confirm key numeric fields make sense.

# Quantity must be >= 1 (no zero/negative sales)
bad_qty = sales[sales["quantity"] <= 0]

# Discount should be between 0 and 1 (0% to 100%)
bad_discount = sales[(sales["discount"] < 0) | (sales["discount"] > 1)]

# Returned should be only 0 or 1 (binary)
bad_returned = sales[~sales["returned"].isin([0, 1])]

print("Bad quantity rows:", len(bad_qty))
print("Bad discount rows:", len(bad_discount))
print("Bad returned rows:", len(bad_returned))


Bad quantity rows: 0
Bad discount rows: 0
Bad returned rows: 0


In [237]:
# Handle Missing Discount (Fill with 0)
# HANDLE MISSING DISCOUNT
# =========================
# If discount is missing, assume no discount (0).
# This is common and avoids losing many rows.

sales["discount"] = sales["discount"].fillna(0)

# Confirm discount missing values are now 0
sales["discount"].isna().sum()

0

In [238]:
# Handle Missing customer_id
# customer_id is required for joining to customers and for customer-level revenue.
# Rows missing customer_id cannot be reliably analyzed for customer behavior.

missing_customer = sales["customer_id"].isna().sum()
print("Missing customer_id before:", missing_customer)

sales = sales.dropna(subset=["customer_id"])

print("Missing customer_id after:", sales["customer_id"].isna().sum())
print("Sales shape after drop:", sales.shape)

Missing customer_id before: 1844
Missing customer_id after: 0
Sales shape after drop: (48156, 8)


In [239]:
# SALES MISSING VALUES AFTER CLEANING
# =========================
# Confirm what missing values remain and where.

sales.isna().sum()

transaction_id    0
date              0
product_id        0
store_id          0
customer_id       0
quantity          0
discount          0
returned          0
dtype: int64

In [240]:
# PRODUCTS CLEANING 
# =========================
# Some datasets use placeholders like "???" for category.
# We replace obvious placeholders with NaN so we can handle them consistently.

product["category"] = product["category"].replace(["???", "UNKNOWN", "Unknown"], np.nan)
product["category"].isna().sum()


499

In [241]:
# JOIN VALIDATION: SALES + PRODUCTS (indicator)
# =========================
# We check if every product_id in sales exists in products.
# indicator=True creates a _merge column:
# - both: matched
# - left_only: sales has product_id not found in products

sales_products_check = sales.merge(
    product,
    on="product_id",
    how="left",
    indicator=True
)

sales_products_check["_merge"].value_counts()

_merge
both          47962
left_only       194
right_only        0
Name: count, dtype: int64

In [242]:
# Inspect Unmatched Product IDs (left_only)
#  INSPECT  UNMATCHED PRODUCT_IDs

# These rows won't have product info like list_price.
# They can affect revenue calculations.

unmatched_products = sales_products_check[sales_products_check["_merge"] == "left_only"]
unmatched_products[["transaction_id", "product_id"]].head(), unmatched_products.shape


(     transaction_id product_id
 74         T0000080    P999999
 814        T0000848    P999999
 1216       T0001271    P999999
 1410       T0001471    P999999
 1491       T0001556    P999999,
 (194, 16))

In [243]:
# MERGE SALES + PRODUCTS (ENRICH SALES)
# =========================
# This creates a transaction-level dataset with product pricing.

merged = sales.merge(
    product,
    on="product_id",
    how="left"
)

merged.shape

(48156, 15)

In [244]:
# Creatating Discounted Price + Revenue
# discounted_price = list_price * (1 - discount)
# revenue = quantity * discounted_price

merged["discounted_price"] = merged["list_price"] * (1 - merged["discount"])
merged["revenue"] = merged["quantity"] * merged["discounted_price"]

merged[["list_price", "discount", "discounted_price", "quantity", "revenue"]].head()

,list_price,discount,discounted_price,quantity,revenue
0,168.10,0.0,168.10,3,504.30
1,121.79,0.0,121.79,1,121.79
2,83.97,0.0,83.97,4,335.88
3,136.73,0.0,136.73,3,410.19
4,23.31,0.0,23.31,4,93.24


In [245]:
# Create Month Columns (for grouping)
# TIME FEATURES: MONTH
# month as period (e.g., 2023-08) is perfect for monthly analysis.


merged["month"] = merged["date"].dt.to_period("M")
merged["month_num"] = merged["date"].dt.month

merged[["date", "month", "month_num"]].head()

,date,month,month_num
0,2023-08-02,2023-08,8
1,2020-08-16,2020-08,8
2,2020-02-21,2020-02,2
3,2024-02-27,2024-02,2
4,2021-07-17,2021-07,7


In [246]:
# MONTHLY CUSTOMER REVENUE

# Sum revenue per customer per month.

monthly_customer_revenue = (
    merged.groupby(["month", "customer_id"])["revenue"]
          .sum()
          .reset_index()
          .sort_values(["month", "revenue"], ascending=[True, False])
)

monthly_customer_revenue.head(10)

,month,customer_id,revenue
333,2020-01,C010688,947.88
95,2020-01,C003198,796.36
466,2020-01,C014915,788.36
352,2020-01,C011214,776.92
349,2020-01,C011182,771.72
765,2020-01,C023201,768.76
833,2020-01,C024846,767.36
798,2020-01,C023800,757.96
531,2020-01,C016690,754.84
714,2020-01,C021765,752.80


In [247]:
# MONTHLY TOTAL REVENUE

monthly_revenue = (
    merged.groupby("month")["revenue"]
          .sum()
          .reset_index()
          .sort_values("month")
)

monthly_revenue

,month,revenue
0,2020-01,203821.815
1,2020-02,181284.061
2,2020-03,199734.659
3,2020-04,198243.905
4,2020-05,194536.994
5,2020-06,198510.809
6,2020-07,219563.198
7,2020-08,203824.769
8,2020-09,185084.867
9,2020-10,214347.685


In [248]:

# TOP CUSTOMERS OVERALL

top_customers = (
    merged.groupby("customer_id")["revenue"]
          .sum()
          .sort_values(ascending=False)
          .head(10)
)

top_customers

customer_id
C019556    3321.532
C011508    3290.434
C014414    3267.740
C009879    3079.862
C002855    2997.714
C007084    2950.976
C023848    2907.786
C020976    2860.913
C018135    2827.348
C017196    2790.560
Name: revenue, dtype: float64

In [249]:
# REVENUE BY PRODUCT CATEGORY

category_revenue = (
    merged.groupby("category")["revenue"]
          .sum()
          .sort_values(ascending=False)
)

category_revenue

category
Shoes          2392352.705
Accessories    2392145.717
Dresses        2383116.291
Tops           2340615.263
Bottoms        2333805.721
Name: revenue, dtype: float64

In [250]:
# Validate Sales → Customers Join
# JOIN VALIDATION: SALES + CUSTOMERS

sales_customer_check = sales.merge(
    customer,
    on="customer_id",
    how="left",
    indicator=True
)

sales_customer_check["_merge"].value_counts()

_merge
both          48156
left_only         0
right_only        0
Name: count, dtype: int64

In [251]:
# Merge Everything (Final Dataset)
# FINAL MERGE: SALES + PRODUCTS + CUSTOMERS + STORES

final_df = (
    sales.merge(product, on="product_id", how="left")
         .merge(customer, on="customer_id", how="left")
         .merge(store, on="store_id", how="left")
)

# Add revenue metrics again (safe if not already computed here)
final_df["date"] = pd.to_datetime(final_df["date"], errors="coerce")
final_df["discount"] = final_df["discount"].fillna(0)
final_df["discounted_price"] = final_df["list_price"] * (1 - final_df["discount"])
final_df["revenue"] = final_df["quantity"] * final_df["discounted_price"]
final_df["month"] = final_df["date"].dt.to_period("M")

final_df.shape

(48156, 25)

In [252]:
# FINAL DATASET PREVIEW

final_df.head()

,transaction_id,date,product_id,store_id,customer_id,quantity,discount,returned,category,color,...,age,gender,city,email,store_name,region,store_size_m2,discounted_price,revenue,month
0,T0000001,2023-08-02,P004681,S004,C010043,3,0.0,0,Shoes,NaN,...,50,Male,Porto,user10043@example.com,Online,Online,950.0,168.10,504.30,2023-08
1,T0000002,2020-08-16,P006662,S003,C022472,1,0.0,0,Accessories,Blue,...,22,Other,Lisbon,user22472@example.com,Faro Outlet,Algarve,336.0,121.79,121.79,2020-08
2,T0000003,2020-02-21,P043402,S005,C016135,4,0.0,0,Shoes,Red,...,19,Other,Braga,user16135@example.com,Coimbra Boutique,Coimbra,238.0,83.97,335.88,2020-02
3,T0000004,2024-02-27,P029875,S003,C005605,3,0.0,1,Shoes,Blue,...,69,Male,Coimbra,user5605@example.com,Faro Outlet,Algarve,336.0,136.73,410.19,2024-02
4,T0000005,2021-07-17,P002476,S003,C005857,4,0.0,0,Tops,Blue,...,33,Other,Porto,user5857@example.com,Faro Outlet,Algarve,336.0,23.31,93.24,2021-07


In [ ]:
# Summary
# - Loaded all datasets successfully
# - Validated structure, datatypes, and missing values
# - Cleaned sales table: fixed date, discount, customer_id
# - Validated join keys using indicator merges
# - Merged sales with dimension tables
# - Created discounted_price and revenue
# - Produced monthly revenue + monthly customer revenue outputs
# print("Notebook is cleaned, validated, merged, and analysis-ready")